In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import plotly.express as px


Dataset : https://icapcarbonaction.com/fr

In [14]:
data = pd.read_csv('./data/icap-graph-price-data-2010-01-01-2025-09-13.csv', header=None)

In [ ]:
lvl0 = data.iloc[0].ffill()
lvl1 = data.iloc[1]
df = data.iloc[2:].copy()
df.columns = pd.MultiIndex.from_arrays([lvl0, lvl1])
df = df.dropna(axis=1, how='all')

In [16]:
date_cols = [col for col in df.columns if col[1] == "Date"]
if not date_cols:
    raise ValueError("No 'Date' column found in the first-row headers.")
df["Date"] = pd.to_datetime(df[date_cols].bfill(axis=1).iloc[:, 0], errors="coerce")

In [17]:
records = []
markets = [m for m in df.columns.get_level_values(0).unique() if m != "Unnamed: 0"]
to_remove = ['Date', ' ', np.nan]
markets = [x for x in markets if x not in to_remove]

In [ ]:
def pick_column(columns, patterns):
    for pat in patterns:
        found = [c for c in columns if re.search(pat, c, flags=re.I)]
        if found:
            return found[0]
    return None

for market in markets:
    sub = df[market].copy()
    if sub.dropna(how="all").empty:
        continue

    price_col = pick_column(
        sub.columns.tolist(),
        patterns=[r"^Primary Market$"
                  ]
    )
    if price_col is None:
        continue

    fx_col = pick_column(
        sub.columns.tolist(),
        patterns=[r"USD/?$", r"/USD$"]
    )

    if fx_col is None:
        cur_col = pick_column(sub.columns.tolist(), patterns=[r"Market Currency"])
        if cur_col is not None and (sub[cur_col].astype(str).str.upper() == "USD").any():
            fx_series = 1.0
        else:
            continue
    else:
        fx_series = sub[fx_col].astype(float)

    # Compute USD price
    price_usd = pd.to_numeric(sub[price_col], errors="coerce") * fx_series

    out = pd.DataFrame({
        "Date": df["Date"],
        "Market": market,
        "Price_USD": price_usd
    }).dropna(subset=["Date", "Price_USD"])

    if not out.empty:
        records.append(out)

In [19]:
prices_long = pd.concat(records, ignore_index=True).sort_values(["Market", "Date"])
prices_long.Market.unique()

array(['California Cap-and-Trade Program (download)',
       'European Union Emissions Trading System (from 2019, download)',
       'European Union Emissions Trading System (until 2018)',
       'Korean Emissions Trading System', 'Québec Cap-and-Trade System',
       'United Kingdom Emissions Trading Scheme (download)',
       'Washington Cap-and-Invest Program (download)'], dtype=object)

In [20]:
market_names = {
    'California Cap-and-Trade Program (download)': 'California Cap-and-Trade',
    'China National Emissions Trading System' : 'China national ETS',
    'European Union Emissions Trading System (from 2019, download)' : 'EU ETS',
    'European Union Emissions Trading System (until 2018)' : 'EU ETS',
    'Korean Emissions Trading System' : 'Korean ETS',
    'United Kingdom Emissions Trading Scheme (download)' : 'UK ETS',
    'Washington Cap-and-Invest Program (download)' : 'Washington Cap-and-Invest',
    'Québec Cap-and-Trade System' : 'Québec Cap-and-Trade'
}

prices_long['Market'] = prices_long['Market'].replace(market_names)
prices_long = (
    prices_long
    .groupby(["Market", "Date"], as_index=False)["Price_USD"].mean()
    .sort_values(["Market", "Date"])
)

In [21]:
fig = px.line(
    prices_long,
    x="Date",
    y="Price_USD",
    color="Market",
    # title="ETS Market Prices (USD)",
    labels={
        "Date": "Date",
        "Price_USD": "Price (USD)",
        "Market": "Market"
    }
)

fig.update_layout(
    plot_bgcolor="white",   # inside the plot area
    # paper_bgcolor="white",  # entire figure background
    # legend=dict(
    #     title="Emissions Trading Markets",
    #     orientation="v",
    #     yanchor="top",
    #     y=1,
    #     xanchor="left",
    #     x=1.02  # push legend slightly to the right
    # ),
    legend=dict(
        title="Emissions Trading Markets",
        orientation="h",
        yanchor="top",
        y=-0.25,
        xanchor="left",
        x=0,
        traceorder="normal",
        itemsizing="constant",
    ),
    margin=dict(r=120),
    yaxis=dict(linecolor="LightGray", showgrid=True, gridcolor="LightGray", zeroline=False, range=[0,121]),
    xaxis=dict(linecolor="LightGray", range=[pd.Timestamp("2015-01-01"), pd.Timestamp("2025-01-10")]),
)

fig.show()
